In [1]:
import numpy as np
import argparse
import time
import cv2
import os

# construct the argument parse and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--image", required=True,
#     help="path to input image")
# ap.add_argument("-c", "--confidence", type=float, default=0.5,
#     help="minimum probability to filter weak detections, IoU threshold")
# ap.add_argument("-t", "--threshold", type=float, default=0.3,
#     help="threshold when applying non-maxima suppression")
# args = vars(ap.parse_args())

default_confidence = 0.5
default_threshold = 0.4

labelsPath = 'yolo\\coco.names'
weightsPath = 'yolo\\yolov3.weights'
configPath = 'yolo\\yolov3.cfg'

# read class names from text file 
LABELS = open(labelsPath).read().strip().split("\n")

# generate different colors to represent each label
COLORS = np.random.uniform(0, 255, size=(len(LABELS), 3))

# read the pre trained model on config file
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

# get output layer
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

def draw_bounding_box(image, classID, confidence, x, y, w, h):
    color = COLORS[classID]
    cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
    text = "{}: {:.4f}".format(LABELS[classID], confidence)
    cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
        0.5, color, 2)
    
def object_detection_in_image(image):
    #create input blob
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),
        swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(output_layers)
    
    boxes = []
    confidences = []
    classIDs = []

    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # ignore weak predictions
            if confidence > default_confidence:
                # scale the bounding box coordinates back relative to the size of the image
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    # apply non-maxima suppression
    idices = cv2.dnn.NMSBoxes(boxes, confidences, default_confidence,
        default_threshold)

    if len(idices) > 0:
        for i in idices.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            draw_bounding_box(image, classIDs[i], confidences[i], x, y, w, h)
            


# reading the input frames from the video
cap = cv2.VideoCapture(0)

while 1:
    # reads frames from a camera
    ret, image = cap.read()
    (H, W) = image.shape[:2]
    
    object_detection_in_image(image)
    cv2.imshow('imgage', image)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
#release the object
cap.release()
  
# Close the window
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'